**Sommario**

- [`flask_login_5` - Database con Flask-SQLAlchemy](#flask_login_5---database-con-flask-sqlalchemy)
  - [Struttura del progetto aggiornata](#struttura-del-progetto-aggiornata)
  - [Configurazione del database in `settings.py`](#configurazione-del-database-in-settingspy)
  - [Gestione del database in `models.py`](#gestione-del-database-in-modelspy)
    - [1. **Importazioni e inizializzazione**](#1-importazioni-e-inizializzazione)
    - [2. **Definizione del modello User**](#2-definizione-del-modello-user)
      - [Classe `User`](#classe-user)
      - [Nome della tabella fisica sul DB](#nome-della-tabella-fisica-sul-db)
      - [Colonne della tabella](#colonne-della-tabella)
      - [Esempio di utilizzo del modello `User` per creare un nuovo record](#esempio-di-utilizzo-del-modello-user-per-creare-un-nuovo-record)
    - [3. **Definizione del modello Film**](#3-definizione-del-modello-film)
      - [Esempio di utilizzo del modello `Film` per creare un nuovo record](#esempio-di-utilizzo-del-modello-film-per-creare-un-nuovo-record)
    - [4. **Inizializzazione del database**](#4-inizializzazione-del-database)
      - [Descrizione generale della funzione `init_db()`](#descrizione-generale-della-funzione-init_db)
      - [Creazione delle tabelle](#creazione-delle-tabelle)
      - [Popolamento delle tabelle `User` e `Film`](#popolamento-delle-tabelle-user-e-film)
  - [Usare il DB in `app.py`](#usare-il-db-in-apppy)
    - [5. **Importazioni**](#5-importazioni)
    - [6. **Configurazione Flask**](#6-configurazione-flask)
    - [7. **Inizializzazione istanza SQLAlchemy**](#7-inizializzazione-istanza-sqlalchemy)
    - [8. **Inizializzazione del database**](#8-inizializzazione-del-database)
    - [9. **Route per il login e controllo credenziali**](#9-route-per-il-login-e-controllo-credenziali)
    - [10. **Route per i Film**](#10-route-per-i-film)

# `flask_login_5` - Database con Flask-SQLAlchemy

Esercitazione INF_PR_PY_WB_E08.

In questa esercitazione vedremo come:

- integrare Flask-SQLAlchemy con Flask;

- configurare e inizializzare il database;

- creare modelli per le tabelle `User` e `Film`;

- utilizzare questi modelli per l'autenticazione degli utenti e la visualizzazione dei film. 

[Flask-SQLAlchemy](https://flask-sqlalchemy.palletsprojects.com/) è un'estensione per Flask che aggiunge le funzioni di [Object-Relational Mapping (ORM)](https://it.wikipedia.org/wiki/Object-relational_mapping) utilizzando [SQLAlchemy](https://www.sqlalchemy.org/). Questo permette di interagire con il database usando oggetti Python invece di scrivere query SQL manualmente.

L'uso di un ORM come SQLAlchemy semplifica dunque la gestione del database e rende il codice più leggibile e manutenibile.

## Struttura del progetto aggiornata

In questa sesta fase modifichiamo ancora una volta la struttura del progetto in questo modo:

```text
flask_login_5/
│
├── app.py                     << ADATTATO ALL'ORM
├── models.py                  << RINOMINATO + NUOVA IMPLEMENTAZIONE
├── settings.py                << MODIFICATO
├── database/
│   ├── db.sqlite3
│   ├── films.csv              << NUOVO
│   └── users.csv
├── static/
│   ├── script.js
│   ├── style.css
│   └── imgs/
│       ├── akira.jpg
│       ├── blade-runner.jpg
│       ├── gits.jpg
│       ├── hackers.jpg
│       ├── nirvana.jpg
│       └── shortcut-icon.png  
└── templates/
    ├── base.html
    ├── films.html
    ├── home.html
    ├── login.html
    └── includes/
        ├── flash.html
        ├── footer.html
        ├── head.html
        └── navbar.html
```

## Configurazione del database in `settings.py`

Il file `settings.py` contiene configurazioni cruciali come il percorso al database e ai file CSV per l'inizializzazione delle tabelle Utenti e Film.

Il file `settings.py` appare così:

```python
import os

BASE_DIR = os.path.abspath(os.path.dirname(__file__))

# Percorso assoluto al file del database SQLite
DATABASE = os.path.join(BASE_DIR, 'database', 'db.sqlite3')

# Percorsi assoluti ai file CSV per le tabelle utenti e film
USER_TABLE_CSV = os.path.join(BASE_DIR, 'database', 'users.csv')
FILM_TABLE_CSV = os.path.join(BASE_DIR, 'database', 'films.csv')

# Nomi delle tabelle
USER_TABLE_NAME = 'user'
FILM_TABLE_NAME = 'film'
```

## Gestione del database in `models.py`

Il file `models.py` contiene la definizione dei cosiddetti "modelli" per le tabelle del database e la funzione per inizializzare il database.

La terminologia "modello" è usata in conformità al paradigma [Model-View-Controller (MVC)](https://it.wikipedia.org/wiki/Model-view-controller), ossia Modello-Vista-Controller.

### 1. **Importazioni e inizializzazione**

Importiamo i moduli necessari e configuriamo SQLAlchemy.

```python
import csv
import os
import sys
import logging
from datetime import datetime
from flask_sqlalchemy import SQLAlchemy
from settings import USER_TABLE_NAME, FILM_TABLE_NAME, USER_TABLE_CSV, FILM_TABLE_CSV

db = SQLAlchemy()  # <-- IMPORTANTE!
```

In particolare

- `from flask_sqlalchemy import SQLAlchemy` importa la classe `SQLAlchemy`;

- `db = SQLAlchemy()` crea un'istanza della classe `SQLAlchemy` del pacchetto Flask-SQLAlchemy. Questa istanza viene utilizzata per interagire con il database, e fornisce un'interfaccia ORM (Object-Relational Mapping) che semplifica la gestione delle operazioni di database all'interno dell'applicazione Flask.


### 2. **Definizione del modello User**

Per prima cosa abbiamo bisogno del modello `User` per rappresentare la tabella Utenti che vogliamo venga creata nel database.

Il codice che segue definisce una classe `User` che rappresenta una tabella fisica nel database utilizzando l'ORM fornito da SQLAlchemy.

La tabella nel database verrà chiamata come la stringa contenuta nella costante `USER_TABLE_NAME`.

```python
class User(db.Model):
    __tablename__ = USER_TABLE_NAME
    id = db.Column(db.Integer, primary_key=True)
    login = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(), nullable=False)
    nome = db.Column(db.String(80))
    cognome = db.Column(db.String(80))
    indirizzo = db.Column(db.String(120))
    citta = db.Column(db.String(80))
    tel1 = db.Column(db.String(20))
    tel2 = db.Column(db.String(20))
    email = db.Column(db.String(100))
    data_nascita = db.Column(db.Date)
    data_reg = db.Column(db.Date)
```

#### Classe `User`

La classe `User` è una sottoclasse di `db.Model`, che è la base per tutti i modelli di database in SQLAlchemy.

Definendo una classe come sottoclasse di `db.Model`, SQLAlchemy può mappare automaticamente la classe a una tabella nel database.

```python
class User(db.Model):
    ...
```

#### Nome della tabella fisica sul DB

L'attributo `__tablename__` specifica il nome della tabella fisica che verrà utilizzato per memorizzare gli oggetti `User` nel nel database.

```python
__tablename__ = USER_TABLE_NAME  # 'user'
```

#### Colonne della tabella

Le colonne (o campi) della tabella sono definite come attributi della classe `User`.

Ciascuna colonna è rappresentata da un oggetto speciale creato attraverso la classe `db.Column`.

Per creare un oggetto `db.Column` bisogna passare un oggetto del tipo corrispondente al tipo di dato che la colonna deve memorizzare, ad esempio `db.Integer`, `db.String` o `db.Date`.

Ecco in dettaglio gli elementi che ci servono:

- `db.Column` classe per creare le colonne.

    - Primo argomento:

        - `db.Integer`, `db.String`, `db.Date` classi con cui definiamo i tipi dei dato che ciascuna colonna dovrà memorizzare.

        - `db.String(80)` gestisce stringhe lunghe al massimo 80 caratteri. Se omesso la lunghezza massima è quella del motore DB usato. Nel caso di SQLite, il tipo `TEXT` è limitao a 1 GB.

    - Altri argomenti:

        - `primary_key=True` argomento di `db.Column` per definire la colonna come *Primary Key* (PK).

        - `unique=True` argomento di `db.Column` per definire che i valori inseriti devono essere unici, quindi non possono esserci due record con lo stesso valore in questa colonna.

        - `nullable=False` argomento di `db.Column` per definire la colonna come obbligatoria, quindi non può contenere valori nulli.

#### Esempio di utilizzo del modello `User` per creare un nuovo record

Ecco un esempio di come ora possiamo creare e aggiungere un nuovo Utente al database utilizzando questa classe appena definita:

```python
# Crea un nuovo utente
new_user = User(
    login='johndoe',
    password='securepassword',
    nome='John',
    cognome='Doe',
    indirizzo='1234 Elm Street',
    citta='Springfield',
    tel1='555-1234',
    tel2='555-5678',
    email='johndoe@example.com',
    data_nascita=datetime.date(1990, 1, 1),
    data_reg=datetime.date.today()
)

# Aggiunge il nuovo utente alla sessione del database
db.session.add(new_user)

# Conferma le modifiche nel database
db.session.commit()
```

### 3. **Definizione del modello Film**

Definiamo anche il modello `Film` che rappresenta la tabella dei Film nel database.

```python
class Film(db.Model):
    __tablename__ = FILM_TABLE_NAME
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(), nullable=False)
    image = db.Column(db.String(), nullable=False)
```

#### Esempio di utilizzo del modello `Film` per creare un nuovo record

Ora possiamo creare un nuovo Film e inserirlo nel DB in questo modo:

```python
# Crea un nuovo utente
new_film = Film(
    title='Matrix',
    image='matrix.jpg',
)

# Aggiunge il nuovo utente alla sessione del database
db.session.add(new_film)

# Conferma le modifiche nel database
db.session.commit()
```

### 4. **Inizializzazione del database**

#### Descrizione generale della funzione `init_db()`

Come leggiamo nella docstring, la funzione `init_db()` inizializza il database e popola le tabelle User e Film con i dati presenti nei file CSV corrispondenti.

In altre parole la funzione:

1. prova a inizializzare il database creando le tabelle se queste non esistono;

2. se la tabella User non è già popolata, la popola;

3. se la tabella Film non è già popolata, la popola.

Ecco come potrebbe apparire la funzione:

```python
def init_db(app):
    """
    Funzione per inizializzare il database e popolare le tabelle con i dati
    presenti nei file CSV.
    :param app: Applicazione Flask corrente (per accedere al logger)
    """
    # CREA LE TABELLE
    db.create_all()
    
    # POPOLA USERS
    if not User.query.first():
        if os.path.exists(USER_TABLE_CSV):
            with open(USER_TABLE_CSV, 'r') as file:
                csv_reader = csv.DictReader(file)
                for row in csv_reader:
                    new_record = User(
                        login=row['LOGIN'],
                        password=row['PASSWORD'],
                        nome=row['NOME'],
                        cognome=row['COGNOME'],
                        indirizzo=row['INDIRIZZO'],
                        citta=row['CITTA'],
                        tel1=row['TEL1'],
                        tel2=row['TEL2'],
                        email=row['EMAIL'],
                        data_nascita=datetime.strptime(row['DATANASCITA'], '%Y-%m-%d'),
                        data_reg=datetime.strptime(row['DATAREG'], '%Y-%m-%d')
                    )
                    db.session.add(new_record)
                db.session.commit()
                app.logger.info(f'Tabella "user" popolata correttamente.')
        else:
            app.logger.error(f'Il file "user.csv" non esiste.')
            sys.exit(1)
    else:
        app.logger.info(f'Tabella "user" già popolata.')

    # POPOLA FILMS
    if not Film.query.first():
        if os.path.exists(FILM_TABLE_CSV):
            with open(FILM_TABLE_CSV, 'r') as file:
                csv_reader = csv.DictReader(file)
                for row in csv_reader:
                    new_record = Film(
                        title=row['TITLE'],
                        image=row['IMAGE']
                    )
                    db.session.add(new_record)
                db.session.commit()
                app.logger.info(f'Tabella "film" popolata correttamente.')
        else:
            app.logger.error(f'Il file "film.csv" non esiste.')
            sys.exit(1)
    else:
        app.logger.info(f'Tabella "film" già popolata.')
```


#### Creazione delle tabelle

La funzione `db.create_all()` crea tutte le tabelle nel database se non esistono già, utilizzando i modelli definiti (`User` e `Film`).

```python
db.create_all()
```

#### Popolamento delle tabelle `User` e `Film`

L'algoritmo di popolamento è uguale per `User` e `Film`. L'unica cosa che cambiano sono i nomi dei campi e i loro valori.

Vediamo la procedura su un ipotetico modello chiamato `MioModello`. In questo caso avremo:

```python
if not MioModello.query.first():
    if os.path.exists('mio_modello.csv'):
        with open('mio_modello.csv', 'r') as csv_file:
            csv_reader = csv.DictReader(csv_file)
            for row in csv_reader:
                new_record = MioModello(
                    campo1=row['CAMPO1'],
                    campo2=row['CAMPO2'],
                    ...
                )
                db.session.add(new_record)
            db.session.commit()
            app.logger.info(f'Tabella popolata correttamente.')
    else:
        app.logger.error(f'Il file CSV non esiste.')
        sys.exit(1)
else:
    app.logger.info(f'Tabella già popolata.')
```

Ecco cosa avviene, passo passo:

1. **Verifica dell'esistenza della tabella**:
    Il metodo `.query.first()` ottiene il primo record esistente nella tabella (senza applicare filtri). Se viene ottenuto `None`, vuol dire che la tabella è vuota. Se la tabella è già popolata il viene scritto un log.
    ```python
    if not MioModello.query.first():
        ...
    else:
        app.logger.info(f'Tabella già popolata.')
    ```

2. **Verifica dell'esistenza del file CSV**: 
    L'istruzione `os.path.exists(MIO_MODELLO_CSV)` verifica se il path al CSV esiste o no. In caso negativo scrive un log e il programma termina.
    ```python
    if os.path.exists('mio_modello.csv'):
        ...
    else:
        app.logger.error(f'Il file CSV non esiste.')
        sys.exit(1)
    ```

3. **Lettura del file CSV**:
    Dentro un contesto `with` viene aperto il file CSV e poi la classe `csv.DictReader()` viene usata per ottenere il file CSV sotto forma di un dizionario.
    ```python
    with open('mio_modello.csv', 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        ...
    ```

4. **Ciclo `for` e creazione di molteplici oggetti-record**:
    Per ogni riga nel CSV, viene creato un oggetto `User` con i dati della riga.
    ```python
    for row in csv_reader:
        new_record = MioModello(  # Creazione nuovo record
            campo1=row['CAMPO1'],
            campo2=row['CAMPO2'],
            ...
        )
        ...
    ```

5. **Aggiunta degli oggetti al database**:
    L'istruzione `db.session.add(new_record)` aggiunge l'oggetto `new_record` alla sessione (transazione) corrente del database.
    ```python
    for row in csv_reader:
        new_record = ...

        db.session.add(new_record)  # Aggiunta del nuovo record alla sessione DB
    ```

6. **Commit delle modifiche**:
    Alla fine del ciclo `for`, si esegue il commit delle modifiche nel database con l'istruzione `db.session.commit()`. Questo serve per confermare che i nuovi dati vengano effettivamente salvati.
    ```python
    for row in csv_reader:
        new_record = ...
        db.session.add(new_record)

    db.session.commit()  # Salvataggio effettivo dei dati sul DB
    ```

7. **Log di successo**: Infine viene registrato un messaggio di log con `app.logger.info` in caso la tabella sia stata popolata con successo.

## Usare il DB in `app.py`

Questo file contiene la logica dell'applicazione Flask, compresa la gestione delle rotte, l'autenticazione degli utenti e la visualizzazione dei film.

### 5. **Importazioni**

Importiamo gli oggetti necessari dai moduli `settings` e `models`:

```python
from flask import Flask, ...
from settings import DATABASE
from models import init_db, db, User, Film
...
```

### 6. **Configurazione Flask**

Dato che dobbiamo modificare la configurazione di Flask `app.config`, rimuoviamo l'istruzione `app.secret_key = 'my_very_secret_key123'` e impostiamo la chiave segreta nel seguente modo, assieme ad altri due nuovi argomenti.

```python
app.config.update(
    SECRET_KEY='my_very_secret_key123',
    SQLALCHEMY_DATABASE_URI='sqlite:///'+DATABASE,
    DEBUG=True
)
```

- `SECRET_KEY='...'` sostituisce `app.secret_key = 'my_very_secret_key123'`.

- `SQLALCHEMY_DATABASE_URI='sqlite:///'+DATABASE` indica il percorso al file del DB.

- `DEBUG=True` sostituisce l'argomento `debug=True` in `app.run()` al fondo del file.

### 7. **Inizializzazione istanza SQLAlchemy**

L'istruzione `db.init_app(app)` inizializza l'istanza di SQLAlchemy (`db`) con l'applicazione Flask (`app`).

```python
db.init_app(app)  # <-- IMPORTANTE!
```


### 8. **Inizializzazione del database**

Inizializziamo le tabelle subito prima dell'avvio dell'applicazione con la funzione `init_db()` importata da `models`:

```python
if __name__ == '__main__':
    with app.app_context():
        # Utilizziamo la funzione creata in model.py per inizializzare le tabelle
        init_db(app)
    app.run()
```

Il blocco `with app.app_context():` assicura che l'istruzione `init_db(app)` sia eseguita all'interno del contesto dell'applicazione. Il motivo è spiegato [nella documentazione ufficiale](https://flask.palletsprojects.com/en/3.0.x/appcontext/#manually-push-a-context).

### 9. **Route per il login e controllo credenziali**

Definiamo la route per il login, che ottiene l'utente e la password dal DB utilizzando i metodi SQLAlchemy:

```python
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        rx_username = request.form.get('tx_user')
        rx_password = request.form.get('tx_password')

        # Utilizziamo il metodo .filter_by() per cercare l'utente nel database
        user = User.query.filter_by(login=rx_username).first()

        # Leggo la password direttamente come attributo di user
        if user and user.password == rx_password:
            ...
        else:
            ...
    return render_template('login.html')
```

Analizziamo l'istruzione `User.query.filter_by(login=rx_username).first()`:

- `User` è la classe che gestisce il modello (tabella) degli Utenti.

- `.query` è una proprietà-classe che crea un oggetto di tipo query per il modello.

- `.filter_by()` è un metodo degli oggetti query che applica un filtro alla query (condizione `WHERE`).
    - `login=rx_username` è un argomento che indica la condizione del filtro. In questo caso si chiede di cercare gli Utenti il cui campo `login` è uguale alla password rivevuta dal client.

- `.first()` è un metodo degli oggetti query che restituisce il primo record di una query oppure `None` se non sono stati trovati record.

Infine, accedere alla password è molto semplice. Se `user` è stato trovato:

- `user.password` ci consente di ottenere la password dell'Utente dal database semplicemente leggendo l'attributo `.password` dell'oggetto-riga `user`, risultato della query precedente.

### 10. **Route per i Film**

Dato che ora anche i Film risiedono sul database, rivediamo la route per la visualizzazione dei Film, utilizzando SQLAlchemy per recuperare i dati dal DB.

```python
@app.route('/films')
def films():
    if 'username' in session:

        # Faccio una query che mi restituisce tutti i Film nella tabella come
        films = Film.query.all()  # list
        
        return render_template('films.html', films=films)
    else:
        return redirect(url_for('login'))
```

L'istruzione `Film.query.all()` esegue la query `.all()`, che restituisce i risultati rappresentati dalla query sotto forma di lista.

Infine passiamo la lista con i Film alla funzione `render_template()` come una qualunque altra struttura dati.